<a href="https://colab.research.google.com/github/dmbk/Anomaly-Detection-System/blob/master/ConvLSTM_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imageio
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
from tensorflow.keras import layers
import time

from IPython import display

from os.path import join
from os import listdir
from os.path import isfile, join, isdir

#import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Reshape, LeakyReLU, Conv2DTranspose, Conv3DTranspose, ConvLSTM2D, BatchNormalization, LayerNormalization, TimeDistributed, Conv2D, Conv3D, ZeroPadding3D, MaxPooling2D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model, Model

import matplotlib.pyplot as plt
import argparse
from os.path import dirname
import cv2
from google.colab.patches import cv2_imshow


tf.keras.backend.set_floatx('float32')

TensorFlow 2.x selected.


In [0]:
class Config:
    def __init__(self, data_dir):
        self.DATASET_PATH = join(data_dir,"UCSDped1/Train/")
        self.TEST_DIR = join(data_dir,"UCSDped1/Test/")
        self.BATCH_SIZE = 2
        self.EPOCHS = 25
        self.GEN_MODEL_PATH = join(data_dir,"model_gen_ConvLSTM_GAN.hdf5")
        self.DIS_MODEL_PATH = join(data_dir,"model_dis_ConvLSTM_GAN.hdf5")
        self.GAN_MODEL_PATH = join(data_dir,"model_combined_ConvLSTM_GAN.hdf5")
        self.dim1 = 10
        self.dim2 = 256
        self.dim3 = 256
        self.dim4 = 1
        self.r_alpha = 0.00000000000001

conf = Config(data_dir="/content/drive/My Drive/UCSD_Anomaly_Dataset.v1p2/") 
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass

In [0]:
def get_clips_by_stride(stride, frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    stride : int
        The distance between two consecutive frames
    frames_list : list
        A list of sorted frames of shape 256 X 256
    sequence_size: int
        The size of the lstm sequence
    Returns
    -------
    list
        A list of clips , 32 frames each
    """
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

def get_clips_list(seq_size):
    """
    seq_size :int 
        The sequence size of individual clip
    Returns
    -------
    list
        A list of training sequences of shape (NUMBER_OF_SEQUENCES,SINGLE_SEQUENCE_SIZE,FRAME_WIDTH,FRAME_HEIGHT,1)
    """
    clips = []
    # loop over the training folders (Train000,Train001,..)
    for f in sorted(listdir(conf.DATASET_PATH)):
        directory_path = join(conf.DATASET_PATH, f)
        if isdir(directory_path):
            all_frames = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(directory_path)):
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    img = Image.open(img_path).resize((256, 256))

                    img = np.array(img, dtype=np.float32) / 256.0
                    all_frames.append(img)
            # get the 32-frames sequences from the list of images after applying data augmentation
            for stride in range(1, 4):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=seq_size))
    return clips


def get_single_test(single_test_path, sz):
    test = np.zeros(shape=(sz, conf.dim2, conf.dim3, conf.dim4))
    cnt = 0
    for f in sorted(listdir(single_test_path)):
        if str(join(single_test_path, f))[-3:] == "tif":
            img = Image.open(join(single_test_path, f)).resize((conf.dim2, conf.dim3))
            #cv2_imshow(np.array(img,dtype=np.float32))
            #cv2.waitKey(0)
            img = np.array(img, dtype=np.float32) / 256
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

def evaluate(test_case_dir, model, sz, gen_only):

    test = get_single_test(join(conf.TEST_DIR,test_case_dir), sz)
    print("Test case loaded")
    sz = test.shape[0] - conf.dim1
    sequences = np.zeros((sz, conf.dim1, conf.dim2, conf.dim3, conf.dim4))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((conf.dim1, conf.dim2, conf.dim3, conf.dim4))
        for j in range(0, conf.dim1):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip

    # get the reconstruction cost of all the sequences
    reconstructed_sequences, sr = model.predict(sequences,batch_size=conf.BATCH_SIZE)
    
    if gen_only == 1:

        for i in range(0, sz):
            cv2_imshow(np.reshape(reconstructed_sequences[i][0],(256, 256))*256)
            cv2.waitKey()

        #reconstruction_shape = (sz,10, 256, 256, 1)
        #sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0,sz)])
        #sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / np.max(sequences_reconstruction_cost)
        #sr = 1.0 - sa
    #print(sr.shape())

    plt.plot(sr)
    plt.ylabel('regularity score Sr(t)')
    plt.xlabel('frame t')
    plt.show()

In [5]:

def get_generator():
    #if reload_model == True and os.path.isfile(conf.GEN_MODEL_PATH):
    #    model=load_model(conf.GEN_MODEL_PATH,custom_objects={'LayerNormalization': LayerNormalization})
    #    return model, True
    print("Loading generator model")
    seq = Sequential()
    seq.add(TimeDistributed(Conv2D(32, (11, 11), strides=(4,4), padding="same"), batch_input_shape=(None, conf.dim1, conf.dim2, conf.dim3, conf.dim4)))   
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(16, (5, 5), strides=(2,2), padding="same")))
    seq.add(LayerNormalization())

    # # # # #
    seq.add(ConvLSTM2D(32,(3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(16, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    # # # # #

    seq.add(TimeDistributed(Conv2DTranspose(16, (5, 5), strides=(2,2), padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2DTranspose(32, (11, 11), strides=(4,4), padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(1, (11, 11), activation="sigmoid", padding="same")))
    #optimizer = tf.keras.optimizers.RMSprop(lr=0.002, clipvalue=1.0, decay=1e-8)
    #seq.compile(optimizer=optimizer, loss='binary_crossentropy')
    seq.summary(line_length=150)
    return seq


def get_discriminator():

    #if reload_model == True and os.path.isfile(conf.DIS_MODEL_PATH):
    #    model=load_model(conf.DIS_MODEL_PATH,custom_objects={'LayerNormalization': LayerNormalization})
    #    return model, True
    model = Sequential()

    # 1st layer group
    model.add(Conv3D(16, (3, 3, 3), activation="relu",name="conv1", 
                     batch_input_shape=(None, conf.dim1, conf.dim2, conf.dim3, conf.dim4),
                     strides=(1, 1, 1), padding="same"))  
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid"))

    # 2nd layer group  
    model.add(Conv3D(16, (3, 3, 3), activation="relu",name="conv2", 
                     strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"))

    # 3rd layer group   
    model.add(Conv3D(32, (3, 3, 3), activation="relu",name="conv3a", 
                     strides=(1, 1, 1), padding="same"))
    model.add(Conv3D(32, (3, 3, 3), activation="relu",name="conv3b", 
                     strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(4, 4, 4), strides=(2, 2, 2), name="pool3", padding="valid"))

    # 4th layer group  
    #model.add(Conv3D(64, (3, 3, 3), activation="relu",name="conv4a", 
    #                 strides=(1, 1, 1), padding="same"))   
    #model.add(Conv3D(64, (3, 3, 3), activation="relu",name="conv4b", 
    #                 strides=(1, 1, 1), padding="same"))
    #model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool4", padding="valid"))



    model.add(Flatten())
                     
    # FC layers group
    #model.add(Dense(512, activation='relu', name='fc6'))
    #model.add(Dropout(.5))
    model.add(Dense(128, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='softmax', name='fc8'))
    model.add(Dropout(.5))
    model.add(Dense(1, activation='sigmoid'))
   
    model.summary(line_length=150)
    return model

def build_model(model, image_dims):
    input_img = Input(shape=image_dims)
    output = model(input_img)
    return Model(input_img,output)





#https://medium.com/analytics-vidhya/implementing-a-gan-in-keras-d6c36bc6ab5f   
#https://www.dlology.com/blog/how-to-do-novelty-detection-in-keras-with-generative-adversarial-network/
#https://arxiv.org/pdf/1802.09088.pdf
#get_generator()
get_discriminator()

Model: "sequential"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                                       Output Shape                                                Param #                
conv1 (Conv3D)                                                     (None, 10, 256, 256, 16)                                    448                    
______________________________________________________________________________________________________________________________________________________
pool1 (MaxPooling3D)                                               (None, 10, 128, 128, 16)                                    0                      
______________________________________________________________________________________________________________________________________________________
conv2 (Conv3D)                                                     (None, 

In [0]:
class ModelContainer:
  def __init__(self, y0, y1, y2):
     self.generator = y0
     self.discriminator = y1
     self.gan = y2

def compile_gan(generator, discriminator):
    image_dims = [conf.dim1, conf.dim2, conf.dim3, conf.dim4]
    optimizer = tf.keras.optimizers.RMSprop(lr=0.002, clipvalue=1.0, decay=1e-8)

    built_dis =  build_model(discriminator, image_dims)

    built_dis.compile(optimizer=optimizer, loss='binary_crossentropy')

    built_gen = build_model(generator, image_dims)
    img = Input(shape=image_dims)

    reconstructed_img = built_gen(img)

    built_dis.trainable = False
    validity = built_dis(reconstructed_img)

    gan_model = Model(img, [reconstructed_img, validity])
    gan_model.compile(loss=['mse', 'binary_crossentropy'],
    loss_weights=[conf.r_alpha, 1],
    optimizer=optimizer)

    return ModelContainer(built_gen, built_dis, gan_model)
    
def train_step(models, batch_clips):
    batch_noise_clips = np.multiply(batch_clips*256, np.random.normal(0, 1, size=(conf.BATCH_SIZE, conf.dim1, conf.dim2, conf.dim3, conf.dim4)))/256
    batch_fake_clips = models.generator.predict_on_batch(batch_noise_clips)

    d_loss_real = models.discriminator.train_on_batch(batch_clips, np.ones(shape=(conf.BATCH_SIZE,1)))
    d_loss_fake = models.discriminator.train_on_batch(batch_fake_clips, np.zeros(shape=(conf.BATCH_SIZE,1)))

    models.gan.train_on_batch(batch_noise_clips, batch_clips, np.ones(shape=(conf.BATCH_SIZE,1)))
    g_loss  = models.gan.train_on_batch(batch_noise_clips, batch_clips, np.ones(shape=(conf.BATCH_SIZE,1)))
    return  g_loss

def train():
    discriminator = get_discriminator()
    generator = get_generator()
    models = compile_gan(generator, discriminator)
    models.gan.summary()

    train_dataset = tf.data.Dataset.from_tensor_slices(np.array(get_clips_list(conf.dim1))).batch(conf.BATCH_SIZE)
    for epoch in range(conf.EPOCHS):
        for batch in train_dataset:
            
            [total_weighted_loss, reconstruction_loss, fooling_loss] = train_step(models, batch)
            print(f'Epoch: {epoch} \t Discriminator Loss: {fooling_loss} \t\t Generator Loss: {reconstruction_loss} \t\t Total Loss: {total_weighted_loss}')
        #gan.reset_states()
        
    
    models.generator.save(conf.GEN_MODEL_PATH,save_format='h5')
    models.discriminator.save(conf.DIS_MODEL_PATH,save_format='h5')
    models.gan.save(conf.GAN_MODEL_PATH, save_format='h5')
    return gan

In [0]:

if os.path.isfile(conf.GAN_MODEL_PATH):
    model=load_model(conf.GAN_MODEL_PATH)
    #,custom_objects={'LayerNormalization': LayerNormalization})
else :
    model = train()

evaluate("Test002", model, 200, 1)

Model: "sequential_1"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                                       Output Shape                                                Param #                
conv1 (Conv3D)                                                     (None, 10, 256, 256, 16)                                    448                    
______________________________________________________________________________________________________________________________________________________
pool1 (MaxPooling3D)                                               (None, 10, 128, 128, 16)                                    0                      
______________________________________________________________________________________________________________________________________________________
conv2 (Conv3D)                                                     (None